In [ ]:
import os
import glob

import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import io

from tensorflow import keras
import tensorflow as tf

# gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.6)
# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
TRAIN_DIR = '../Dataset/training_set/training_set/'
TEST_DIR = '../Dataset/test_set/test_set/'

items = ['cats', 'dogs']
SIZE = (120, 120, )

X_train = []
X_test = []
Y_train = []
Y_test = []


In [ ]:
'''
Reading Training Data
'''
cats = []
dogs = []

for e in items:
    _dir = os.path.join(TRAIN_DIR, e)

    for path in glob.glob(_dir + '/*.jpg'):
        img = io.imread(path, as_gray= True)
        img = cv2.resize(img, SIZE)
        if e == 'cats':
            cats.append(img)
        else:
            dogs.append(img)
        

In [ ]:
cats = np.array(cats)
dogs = np.array(dogs)

In [ ]:
cats = np.expand_dims(cats, axis= -1)
dogs = np.expand_dims(dogs, axis= -1)

In [ ]:
'''
Data Augmentation
'''
from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator(
    rotation_range= 45,
    height_shift_range= .2,
    width_shift_range= .2,
    zoom_range= .2,
    vertical_flip= True,
    horizontal_flip= True,
    fill_mode= 'reflect',
)
BATCH_LEN = int(1e4)
i = 0

for batch in gen.flow(cats):
    for img in batch:
        X_train.append(img)
    i += 1
    if i >= BATCH_LEN:
        break

In [ ]:
for _ in range(len(cats)):
    Y_train.append(0)

In [ ]:
BATCH_LEN = int(1e4)
i = 0

for batch in gen.flow(dogs):
    for img in batch:
        X_train.append(img)
        Y_train.append(1)
    i += 1
    if i >= BATCH_LEN:
        break

In [ ]:
X_train = np.expand_dims(X_train, axis= -1)
X_test = np.expand_dims(X_test, axis= -1)

In [ ]:
'''
Reading testing data
'''
for e in items:
    _dir = os.path.join(TEST_DIR, e)

    for path in glob.glob(_dir + '/*.jpg'):
        img = io.imread(path, as_gray= True)
        img = cv2.resize(img, SIZE)
        X_test.append(img)
        Y_test.append(items.index(e))
        

X_test = np.array(X_test)
Y_test = np.array(Y_test)


In [ ]:
from sklearn.utils import shuffle

X_train, Y_train = shuffle(X_train, Y_train)
X_test, Y_test = shuffle(X_test, Y_test)

In [ ]:
X_train.shape , X_train.max()

In [ ]:
X_train = tf.expand_dims(X_train, -1)
X_test = tf.expand_dims(X_test, -1)
Y_train = keras.utils.to_categorical(Y_train, 2)
Y_test = keras.utils.to_categorical(Y_test, 2)

In [ ]:
X_train[0]

In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), strides= (1, 1), input_shape= X_train.shape[1:], activation= 'selu'),
    keras.layers.MaxPool2D(pool_size= (3, 3)),

    keras.layers.Conv2D(24, (3, 3), strides= (2, 2), activation= 'selu'),
    keras.layers.MaxPool2D(pool_size= (2, 2)),

    tf.keras.layers.GlobalMaxPool2D(),

    keras.layers.Dense(16, activation= 'relu'),
    keras.layers.Dense(8, activation= 'relu'),
    keras.layers.Dense(2, activation= 'softmax')

])

model.summary()

In [ ]:
model.compile(optimizer= keras.optimizers.Adam(), loss= keras.losses.categorical_crossentropy, metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size= 32, epochs= 10, validation_data= (X_test, Y_test))